In [4]:
import cv2
import os
import numpy as np
import joblib
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# make 
X = []
y = []
for i in os.listdir('D:/Ducat/NLP/NLP_datasets/male-20250726T060959Z-1-001/male'):
    img = cv2.imread(f'D:/Ducat/NLP/NLP_datasets/male-20250726T060959Z-1-001/male/{i}',cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(100,100))
    img = img/255
    X.append(img.flatten())
    y.append('male')

for i in os.listdir('D:/Ducat/NLP/NLP_datasets/female-20250714T150010Z-1-001/female'):
    img = cv2.imread(f'D:/Ducat/NLP/NLP_datasets/female-20250714T150010Z-1-001/female/{i}',cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(100,100))
    img = img/255
    X.append(img.flatten())
    y.append('female')

X = np.array(X)
y = np.array(y)

pca = PCA(n_components=149)
X_new = pca.fit_transform(X)

model_log = LogisticRegression(max_iter=2000)
model_log.fit(X_new,y)

model_face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
img_clr = cv2.imread('wp5075652-virushka-wallpapers.jpg')
img_gray = cv2.cvtColor(img_clr,cv2.IMREAD_GRAYSCALE)
face_list = model_face.detectMultiScale(img_gray,minNeighbors= 9)
for i,j,k,l in face_list:
    face = img_clr[j:j+l,i:i+k]
    face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face,(100,100))
    face = face/255
    face = pca.transform([face.flatten()])
    pred = model_log.predict(face)
    cv2.rectangle(img_clr,(i,j),(i+k,j+l),(0,0,255),2)
    cv2.putText(img_clr,pred[0],(i,j-5),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
img_2 = cv2.resize(img_clr,(500,600))
cv2.imshow('test',img_2)
cv2.waitKey()
cv2.destroyAllWindows()


joblib.dump(model_log,'gender.pkl')
joblib.dump(pca,'pca_transform.pkl')